# Dimensionality Reduction of Pan-Cancer Gene Expression

Here I perform different dimensionality reduction algorithms on TCGA pan-cancer data. There are over 10,000 different tumors measured from 33 different cancer-types.

The purpose of finding distinct representations of the data are to compare how well the aggregate features perform in various tasks. The current algorithms implemented in this script are:

1. PCA
2. ICA
3. NMF
5. MDS
6. Spectral Embedding

I also implement these algorithms for copy number data.

Other implemented algorithms in alternative scripts include

1. Denoising Autoencoder
2. Variational Autoencoder


In [1]:
import os
import pandas as pd

from sklearn import manifold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import decomposition
from statsmodels.robust.scale import mad

## Load RNAseq and Copy Number Data

In [3]:
# Load Z Score Transformed Data
rnaseq_file = 'https://github.com/greenelab/tybalt/raw/928804ffd3bb3f9d5559796b2221500c303ed92c/data/pancan_scaled_rnaseq.tsv.gz'
rnaseq_df = pd.read_table(rnaseq_file, index_col=0)

In [13]:
# Load Zero-One Transformed Data
rnaseq_zeroone_file = 'https://github.com/greenelab/tybalt/raw/928804ffd3bb3f9d5559796b2221500c303ed92c/data/pancan_scaled_zeroone_rnaseq.tsv.gz'
rnaseq_zeroone_df = pd.read_table(rnaseq_zeroone_file, index_col=0)
print(rnaseq_zeroone_df.shape)

(10459, 5000)


In [3]:
copy_file = os.path.join('processed', 'pancan_processed_copynumber.tsv')
copy_df = pd.read_table(copy_file, index_col=0)

## PCA

Principal components analysis (PCA) is a linear dimensionality reduction algorithm that aims to find the highest sources of variation in input data. We use the [sklearn implementation of PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

In [6]:
%%time
pca = decomposition.PCA(n_components=100)
pca_out = pca.fit_transform(rnaseq_df)

CPU times: user 15.6 s, sys: 3.28 s, total: 18.8 s
Wall time: 5.52 s


In [7]:
pca_rnaseq = pd.DataFrame(pca_out, index=rnaseq_df.index)
pca_rnaseq.columns = pca_rnaseq.columns + 1
pca_rnaseq.index.name = 'tcga_id'

In [8]:
pca_rnaseq_file = os.path.join('data', 'pca_rnaseq.tsv.gz')
pca_rnaseq.to_csv(pca_rnaseq_file, sep='\t', compression='gzip')

## ICA

Independent Components Analysis (ICA) looks for maximally linear independent signals in input data. We use the  [sklearn implementation of FastICA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html).

In [10]:
%%time
ica = decomposition.FastICA(n_components=100)
ica_out = ica.fit_transform(rnaseq_df)

CPU times: user 3min 12s, sys: 7.32 s, total: 3min 19s
Wall time: 55.6 s


In [11]:
ica_rnaseq = pd.DataFrame(ica_out, index=rnaseq_df.index)
ica_rnaseq.columns = ica_rnaseq.columns + 1
ica_rnaseq.index.name = 'tcga_id'

In [12]:
ica_rnaseq_file = os.path.join('data', 'ica_rnaseq.tsv.gz')
ica_rnaseq.to_csv(ica_rnaseq_file, sep='\t', compression='gzip')

## NMF

Non-negative matrix factorization (NMF) factorizes a given matrix (X) into two matrices of sample (W) and gene (H) activation across a reduced set of internal components, or latent features. We use the [sklearn implementation of NMF](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html).

In [14]:
%%time
nmf = decomposition.NMF(n_components=100, init='nndsvdar', tol=5e-3)
nmf_out = nmf.fit_transform(rnaseq_zeroone_df)

CPU times: user 1min 52s, sys: 38.4 s, total: 2min 30s
Wall time: 45.5 s


In [15]:
nmf_rnaseq = pd.DataFrame(nmf_out, index=rnaseq_zeroone_df.index)
nmf_rnaseq.columns = nmf_rnaseq.columns + 1
nmf_rnaseq.index.name = 'tcga_id'

In [16]:
nmf_rnaseq_file = os.path.join('data', 'nmf_rnaseq.tsv.gz')
nmf_rnaseq.to_csv(nmf_rnaseq_file, sep='\t', compression='gzip')

## MDS

In [19]:
mds = manifold.MDS(3, max_iter=100, n_init=1)
mds_out = mds.fit_transform(data_subset_df)

In [20]:
mds_out = pd.DataFrame(mds_out, columns=['x', 'y'])
mds_out.index = data_subset_df.index
mds_out.index.name = 'tcga_id'
mds_out.to_csv('mds_out.tsv', sep='\t')

## SE

In [21]:
se = manifold.SpectralEmbedding(2, n_neighbors=10)
se_out = se.fit_transform(data_subset_df)

/home/gway/anaconda3/lib/python3.5/site-packages/sklearn/manifold/spectral_embedding_.py:229: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [22]:
se_out = pd.DataFrame(se_out, columns=['x', 'y'])
se_out.index = data_subset_df.index
se_out.index.name = 'tcga_id'
se_out.to_csv('se_out.tsv', sep='\t')